In [1]:
import mne
import os.path as op
import numpy as np
import pandas as pd
import pickle
import statsmodels.api as sm
import matplotlib.pylab as plt
from joblib import Parallel, delayed

In [2]:
beh_file = "/cubric/scratch/c1557187/energy_landscape/021/resamp_beh.pkl"
stc_file = "/cubric/scratch/c1557187/stirring_source/data/021/filtered_01_40/src_en_dSPM.pickle"

In [3]:
beh = pd.read_pickle(beh_file)
# with open(stc_file, "rb") as input_file:
#         stc = pickle.load(input_file)

In [4]:
beh_data = beh.iloc[1]

In [5]:
beh_data

subject                                                       21
clockwise                                                      1
trial                                                          1
run                                                            0
session                                                        3
conditions                                                     5
comb_cond                                                      0
engage_rt                                                0.82988
change_rt                                             0.00867929
acc_phase1                                              0.530069
acc_phase2                                                 0.999
x              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
y              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
angle          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
radius         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
engage_ix                

In [6]:
stc = mne.read_source_estimate("source_est_1")
stc_data = stc.data

In [7]:
X = beh_data.angle[beh_data.engage_ix-51:beh_data.engage_ix+300]
def ios(ix, X, stc_data):
    Y = stc_data[ix]
    model = sm.OLS(X, Y)
    results = model.fit()
    return (results.rsquared_adj, results.params[0])

results = Parallel(n_jobs=-1, backend="multiprocessing", verbose=0)(delayed(ios)(ix, X, stc_data) for ix in range((stc_data.shape[0])))

In [8]:
results = np.array(results)
r2_adj = results[:,0]
beta = results[:,1]

In [9]:
# stc.data = np.abs(np.array([r2_adj, r2_adj]).transpose())
stc.data = np.array([beta, beta]).transpose()

In [10]:
%matplotlib
stc.plot(hemi="both", subject="fsaverage", time_viewer=True);

Using matplotlib backend: Qt5Agg


In [11]:
r2_adj.min()

0.23305370840465855